In [ ]:
%load_ext autoreload
%load_ext autotime

In [ ]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')

Some logs are stored in VAST with some stored in GPFS.

Set `log_dir` to `/vast1/` when running the notebook on Quartz, otherwise use `/gpfs1/`.

In [ ]:
#log_dir = "/p/gpfs1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/hacc|/nodes-32/workflow-0/_parquet"
#log_dir = "/p/gpfs1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
#log_dir = "/p/gpfs1/iopp/recorder_app_logs/montage_pegasus/nodes-32/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-cosmoflow/nodes-32/_parquet" # FIXME
log_dir = "/p/gpfs1/iopp/recorder_app_logs/mummi-wemul/nodes-32-ppn-32/_parquet"# FIXME
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-jag/nodes-32/_parquet"
#log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"


`RecorderAnalyzer` is the main part that initializes the cluster. It creates `LocalCluster` or `LSFCluster` depending on the `force_local` variable. When it is set to `true`, it will just create a `LocalCluster`. 

In [ ]:
%autoreload 2

from wisio.recorder import VIEW_TYPES, RecorderAnalyzer

recorder_analyzer = RecorderAnalyzer(
    cluster_manager_args=dict(
#         force_local=True,
        cluster_settings=dict(
#             cores=32,
            dashboard_port=3646,
            local_directory="/var/tmp/dask-recorder",
            log_file="%J.log",
#             memory=3200,
            worker_queue="pdebug"
        )
    ),
    working_dir='.recorder',
    debug=True
)

recorder_analyzer


In [ ]:
[client.dashboard_link for client in recorder_analyzer.cluster_manager.clients]


From this point on, every step is exactly the same within the `analyze_parquet` method of `RecorderAnalyzer`.

In [ ]:
%autoreload 2

global_min_max = recorder_analyzer.load_global_min_max(log_dir=log_dir)
global_min_max


In [ ]:
%autoreload 2

from wisio._recorder.analysis import (
    compute_main_view,
    compute_max_io_time,
    compute_view
)

`compute_main_view` automatically does `persist` before returning the DataFrame. You may play with that part if the `main_view` doesn't fit within the allocated memory. That said, `persist`ing it increases performance significantly.

In [ ]:
%autoreload 2

main_view = compute_main_view(
    log_dir=log_dir,
    global_min_max=global_min_max,
    view_types=VIEW_TYPES
)

main_view


`max_io_time` is just a `dd.Scalar` since I tried to compute everyting lazily.

In [ ]:
%autoreload 2

max_io_time = compute_max_io_time(main_view=main_view)
max_io_time

You may configure `delta` and `cut` using the following variables.

In [ ]:
DELTA = 0.0001
CUT = 0.5

The following is how all the views (including `expanded_view` and `bottleneck_view`-I didn't like the `cut_view` name). So, if you want to play with that part `compute_view` is the method you should look for.

In [ ]:
%autoreload 2

import itertools as it

def _view_permutations(r: int):
    return it.permutations(VIEW_TYPES, r + 1)

views = {}
for view_permutation in it.chain.from_iterable(map(_view_permutations, range(len(VIEW_TYPES)))):
    # Compute view
    views[view_permutation] = compute_view(
        main_view=main_view,
        views=views,
        view_permutation=view_permutation,
        max_io_time=max_io_time,
        delta=DELTA,
        cut=CUT
    )


len(views)

The following just initializes the bottleneck detector.

In [ ]:
%autoreload 2

from wisio._recorder.bottlenecks import RecorderBottleneckDetector

bottleneck_detector = RecorderBottleneckDetector(
    logger=recorder_analyzer.logger,
    log_dir=log_dir,
)

bottleneck_detector


This part is where our nested loops lie. The output type is `{view_type: bottleneck_dict}`.

In [ ]:
%autoreload 2

bottlenecks = bottleneck_detector.detect_bottlenecks(
    views=views,
    view_types=VIEW_TYPES,
)

len(bottlenecks)

The following saves the bottlenecks per `view_type`. 

In [ ]:
%autoreload 2

recorder_analyzer.save_bottlenecks(log_dir=log_dir, bottlenecks=bottlenecks)
